<a href="https://colab.research.google.com/github/Shubh8434/code_snippets/blob/master/iitp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.4 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from transformers import RobertaTokenizer, RobertaModel
import torch
import pickle

from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
def extract_feature(tweet):

    inputs = tokenizer(tweet, return_tensors="pt", truncation=True)
    outputs = model(**inputs)

    last_hidden_states = outputs.last_hidden_state
    
    if (last_hidden_states.shape[1]<20):
        pad = torch.zeros((1,20-last_hidden_states.shape[1],768))
        output = torch.cat((last_hidden_states,pad),1)
    else:
        output = last_hidden_states[:,:20,:]
        
    return output.detach().numpy()
    
    #return outputs.pooler_output.detach().numpy()

In [5]:
def convert(tweet):
    tweet = str(tweet).split(' ')
    
    if(len(tweet)<20):
        for i in range(20-len(tweet)):
            tweet.append(' ')
    
    else:
        tweet = tweet[:20]
        
        
    return tweet

In [6]:
def lcs(X, Y, m, n):
  L = [[0 for i in range(n+1)] for j in range(m+1)]

	# Following steps build L[m+1][n+1] in bottom up fashion. Note
	# that L[i][j] contains length of LCS of X[0..i-1] and Y[0..j-1]
  for i in range(m+1):
    for j in range(n+1):
      if i == 0 or j == 0:
        L[i][j] = 0
      elif X[i-1] == Y[j-1]:
        L[i][j] = L[i-1][j-1] + 1
      else:
        L[i][j] = max(L[i-1][j], L[i][j-1])

		# Create a string variable to store the lcs string
  lcs1=[]

	# Start from the right-most-bottom-most corner and
	# one by one store characters in lcs[]
  i = m
  j = n
	
  while i > 0 and j > 0:

		# If current character in X[] and Y are same, then
		# current character is part of LCS
    if X[i-1] == Y[j-1]:
      lcs1.append(i-1)
      i -= 1
      j -= 1

		# If not same, then find the larger of two and
		# go in the direction of larger value
    elif L[i-1][j] > L[i][j-1]:
      i -= 1
    else:
      j -= 1
      
  ans=[]
  
  for i in range(len(X)):
    ans.append(0)
    
  for i in range(len(lcs1)):
    ans[lcs1[i]]=1
    
  if(len(ans)<20):
    for i in range(20-len(ans)):
        ans.append(0)
    
  return ans[:20]


In [7]:
embed = "/content/drive/MyDrive/COLING_Review/word_level_embeddings.csv"

embeddings = pd.read_csv(embed)

embeddings = np.array(embeddings)

embeddings = embeddings[:,1:]

print(embeddings.shape) 

(7468, 768)


In [8]:
sheets_dict = pd.read_csv("/content/drive/MyDrive/Untitled spreadsheet - Hinglish Complaint Dataset_Shubham - Sheet1.csv")
domain_to_ix = {'Book': 0, 'Phone': 1, 'Headphones': 2, 'Smartwatch': 3, 'TV': 4, 'Laptop': 5, 'Jio network': 6, 'Utensils': 7, 'Food': 8, 'Tablet': 9, 'Sewing Machine': 10,'Processor': 11, 'Memory Card': 12, 'Camera': 13, 'Food': 14, 'Grooming': 15, 'book': 16, 'Smartshoes': 17}
senti_to_ix = {'Neutral': 0, 'Positive': 1, 'Negative': 2, 'negative': 2, 'neutral': 0, 'positive': 1}
emoti_to_ix = {'other': 0, 'happiness': 1, 'sadness': 2, 'anger': 3, 'disgust': 4, 'surprise': 5, 'fear': 6} 

In [9]:
vector={}
 
vector["id"]=[]
vector["tweet"]=[]
vector["feat"]=[]
vector["emoti-label"]=[]
vector["senti-label"]=[]
vector["domain-label"]=[]
vector["task-label"]=[]
vector["span"]=[]

In [10]:
sheets_dict['id'] = sheets_dict.index

In [11]:
sheets_dict['Category'].value_counts()

Book              2838
Phone              674
Headphones          48
Smartwatch          43
TV                  31
Laptop              26
Jio network         13
Food                 8
Utensils             6
Sewing Machine       5
Tablet               5
Processor            5
Memory Card          3
Camera               3
Grooming             1
book                 1
Smartshoes           1
Name: Category, dtype: int64

In [12]:
sheets_dict.columns

Index(['Category', 'Label', 'Reviews', 'Sentiment', 'Emotion', 'Cause', 'id'], dtype='object')

In [13]:
cnt=0
length =0


for i in range(len(sheets_dict)):

    print("step : ",i)
    
    x = str(sheets_dict["Reviews"][i]).split(' ')  
    
    
    
    
    
    
    if cnt==0:
        vector["id"] = np.array(sheets_dict["id"][i])
        vector["tweet"] = np.array(convert(sheets_dict["Reviews"][i]))
        vector["feat"] = np.array(extract_feature(sheets_dict["Reviews"][i]))
        vector["emoti-label"] = np.array(emoti_to_ix[sheets_dict["Emotion"][i]])
        vector["senti-label"] = np.array(senti_to_ix[sheets_dict["Sentiment"][i]])
        vector["domain-label"] = np.array(domain_to_ix[sheets_dict["Category"][i]])
        vector["task-label"] = np.array(sheets_dict["Label"][i])
        vector["span"] = np.array(lcs(sheets_dict["Reviews"][i].split(' '),sheets_dict["Cause"][i].split(' '),len(sheets_dict["Reviews"][i].split(' ')),len(sheets_dict["Cause"][i].split(' '))))
        cnt = cnt + 1
    else:
        vector["id"] = np.vstack((vector["id"],np.array(sheets_dict["id"][i])))
        vector["tweet"] = np.vstack((vector["tweet"],np.array(convert(sheets_dict["Reviews"][i]))))
        vector["feat"] = np.vstack((vector["feat"],np.array(extract_feature(sheets_dict["Reviews"][i]))))
        vector["emoti-label"] = np.vstack((vector["emoti-label"],np.array(emoti_to_ix[sheets_dict["Emotion"][i]])))
        vector["senti-label"] = np.vstack((vector["senti-label"],np.array(senti_to_ix[sheets_dict["Sentiment"][i]])))
        vector["domain-label"] = np.vstack((vector["domain-label"],np.array(domain_to_ix[sheets_dict["Category"][i]])))
        vector["task-label"] = np.vstack((vector["task-label"],np.array(sheets_dict["Label"][i])))
        vector["span"] = np.vstack((vector["span"],np.array(lcs(str(sheets_dict["Reviews"][i]).split(' '),str(sheets_dict["Cause"][i]).split(' '),len(str(sheets_dict["Reviews"][i]).split(' ')),len(str(sheets_dict["Cause"][i]).split(' '))))))
        

step :  0
step :  1
step :  2
step :  3
step :  4
step :  5
step :  6
step :  7
step :  8
step :  9
step :  10
step :  11
step :  12
step :  13
step :  14
step :  15
step :  16
step :  17
step :  18
step :  19
step :  20
step :  21
step :  22
step :  23
step :  24
step :  25
step :  26
step :  27
step :  28
step :  29
step :  30
step :  31
step :  32
step :  33
step :  34
step :  35
step :  36
step :  37
step :  38
step :  39
step :  40
step :  41
step :  42
step :  43
step :  44
step :  45
step :  46
step :  47
step :  48
step :  49
step :  50
step :  51
step :  52
step :  53
step :  54
step :  55
step :  56
step :  57
step :  58
step :  59
step :  60
step :  61
step :  62
step :  63
step :  64
step :  65
step :  66
step :  67
step :  68
step :  69
step :  70
step :  71
step :  72
step :  73
step :  74
step :  75
step :  76
step :  77
step :  78
step :  79
step :  80
step :  81
step :  82
step :  83
step :  84
step :  85
step :  86
step :  87
step :  88
step :  89
step :  90
step :  9

In [34]:
print(vector["task-label"].shape)
print("saving data ...")
output=open("/content/drive/MyDrive/COLING_Review/dataset-complain_ss1.pkl","wb")
pickle.dump(vector,output)    

(3711, 1)
saving data ...


In [35]:
%cd /content/drive/MyDrive/COLING_Review

/content/drive/MyDrive/COLING_Review


In [36]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import  mean_absolute_error
from sklearn.model_selection import train_test_split, StratifiedKFold
from pathlib import Path
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from bert_cnn_fusion import bert_span


import os
import pickle


# import EarlyStopping
import os, sys
#sys.path.append('path_to_the_module/early-stopping-pytorch')
from pytorchtools import EarlyStopping
#from torchsample.callbacks import EarlyStopping


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device='cpu'
print(device)



cuda


In [37]:
id = vector["id"]
tweet = vector["tweet"]
feat = vector["feat"]
emoti = vector["emoti-label"]
senti = vector["senti-label"]
domain = vector["domain-label"]
task = vector["task-label"]
span = vector["span"]

In [38]:
vector["senti-label"]

array([[2],
       [0],
       [1],
       ...,
       [1],
       [1],
       [2]])

In [39]:
cnt=0

for i in range(task.shape[0]):
    if(task[i]==1):
        cnt=cnt+1
        
print(cnt)

652


In [40]:
task

array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [41]:
# task =keras.utils.to_categorical(task)
# print(task.shape)

In [42]:
task

array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [43]:
StratifiedKFold()

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

In [44]:
senti

array([[2],
       [0],
       [1],
       ...,
       [1],
       [1],
       [2]])

In [45]:
tweet_train,tweet_test,task_train,task_test=train_test_split(tweet,task,test_size=0.2,stratify=task,random_state=0)
id_train,id_test,task_train,task_test=train_test_split(id,task,test_size=0.2,stratify=task,random_state=0)
feat_train,feat_test,task_train,task_test=train_test_split(feat,task,test_size=0.2,stratify=task,random_state=0)
emoti_train,emoti_test,task_train,task_test=train_test_split(emoti,task,test_size=0.2,stratify=task,random_state=0)
senti_train,senti_test,task_train,task_test=train_test_split(senti,task,test_size=0.2,stratify=task,random_state=0)
domain_train,domain_test,task_train,task_test=train_test_split(domain,task,test_size=0.2,stratify=task,random_state=0)
span_train,span_test,task_train,task_test=train_test_split(span,task,test_size=0.2,stratify=task,random_state=0)


In [46]:
task=task_train


tweet_train,tweet_val,task_train,task_val=train_test_split(tweet_train,task,test_size=0.1,stratify=task,random_state=0)
id_train,id_val,task_train,task_val=train_test_split(id_train,task,test_size=0.1,stratify=task,random_state=0)
feat_train,feat_val,task_train,task_val=train_test_split(feat_train,task,test_size=0.1,stratify=task,random_state=0)
emoti_train,emoti_val,task_train,task_val=train_test_split(emoti_train,task,test_size=0.1,stratify=task,random_state=0)
senti_train,senti_val,task_train,task_val=train_test_split(senti_train,task,test_size=0.1,stratify=task,random_state=0)
domain_train,domain_val,task_train,task_test=train_test_split(domain_train,task,test_size=0.1,stratify=task,random_state=0)
span_train,span_val,task_train,task_val=train_test_split(span_train,task,test_size=0.1,stratify=task,random_state=0)


In [47]:
train={"feat":feat_train,"id":id_train,"label":task_train,"emotion":emoti_train,"sentiment":senti_train, "span":span_train,"domain":domain_train}
val={"feat":feat_val,"id":id_val,"label":task_val,"emotion":emoti_val,"sentiment":senti_val, "span":span_val,"domain":domain_val}
test={"feat":feat_test,"id":id_test,"label":task_test,"emotion":emoti_test,"sentiment":senti_test, "span":span_test,"domain":domain_test}



In [48]:
class Tweet(torch.utils.data.Dataset):
    def __init__(self,data):
        self.data=data
        
    def __len__(self):
        return self.data["label"].shape[0]
    def __getitem__(self,idx):
    
        if(torch.is_tensor(idx)):
            idx=idx.tolist()
            
            
            
        feat=torch.tensor(self.data["feat"][idx].astype(np.float)).float().to(device)
        id=torch.tensor(self.data["id"][idx].astype(np.float)).long().to(device)
        
       
        label=torch.tensor(self.data["label"][idx].astype(np.long)).to(device)
        emotion=torch.tensor(self.data["emotion"][idx]).long().to(device)
        domain=torch.tensor(self.data["domain"][idx]).long().to(device)
        sentiment=torch.tensor(self.data["sentiment"][idx]).long().to(device)
        span=torch.tensor(self.data["span"][idx]).float().to(device)
    
        
        sample = {
            
            "id":id,
            "feat":feat,
            "label":label,
            "domain":domain,
            "emotion":emotion,
            "sentiment":sentiment,
            "span":span,
        }
        
        return sample
        
        

In [49]:
 
tweet_train = Tweet(train)
dataloader_train = DataLoader(tweet_train, batch_size=32,shuffle=False, num_workers=0)

print("train_data loaded")

tweet_val = Tweet(val)
dataloader_val = DataLoader(tweet_val, batch_size=32,shuffle=False, num_workers=0)
print("validation_data loaded")


tweet_test = Tweet(test)
dataloader_test = DataLoader(tweet_test, batch_size=32,shuffle=False, num_workers=0) 


train_data loaded
validation_data loaded


In [50]:
import torch
import torch.nn as nn


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device='cpu'
print(device)

class bert_span(nn.Module):
    def __init__(self,):
        super(bert_span,self).__init__()
        
        self.word_lstm = nn.LSTM(768,256,batch_first=True, bidirectional=True)
        
        self.q_w = nn.Linear(512,512)
        self.k_w = nn.Linear(512,512)
        self.v_w = nn.Linear(512,512)
        
        self.sent_lstm = nn.LSTM(512,256,batch_first = True, bidirectional = True)
        
        #self.conv = nn.Conv1d(768,512,4,stride =4)
        #self.maxpool = nn.MaxPool
        
        self.q_s = nn.Linear(512,512)
        self.k_s = nn.Linear(512,512)
        self.v_s = nn.Linear(512,512)
        
        dropout = 0.5
        
        self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.Tanh()
        self.dropout = nn.Dropout(dropout)
        
        #emotion
        
        self.emo_fc1 = nn.Linear(768,512)
        self.emo_fc2 = nn.Linear(512,256)
        self.emo_out = nn.Linear(256,7)
        
        #sentiment
        
        self.sen_fc1 = nn.Linear(768,512)
        self.sen_fc2 = nn.Linear(512,256)
        self.sen_out = nn.Linear(256,3)
        
        self.comp_fc1 = nn.Linear(768,512)
        self.comp_fc2 = nn.Linear(512,256)
        self.comp_fc3 = nn.Linear(256,128)
        self.comp_out = nn.Linear(128,2)
        
        #self.span_fc1 = nn.Linear(768,512)
        self.span_fc2 = nn.Linear(256,128)
        
        #self.comp_fc = nn.Linear(128,64)
        self.span_out = nn.Linear(128,20)
                
        #attention
        
        self.alphas_m1 = nn.ParameterList([torch.nn.Parameter(torch.rand(1)) for i in range(1)])
        self.alphas_m2 = nn.ParameterList([torch.nn.Parameter(torch.rand(1)) for i in range(1)])
        
        
        
        self.alphas_e = nn.ParameterList([torch.nn.Parameter(torch.rand(1)) for i in range(2)])
        self.alphas_s = nn.ParameterList([torch.nn.Parameter(torch.rand(1)) for i in range(2)])
        self.alphas_c = nn.ParameterList([torch.nn.Parameter(torch.rand(1)) for i in range(2)])
        self.alphas_sp = nn.ParameterList([torch.nn.Parameter(torch.rand(1)) for i in range(2)])
        
        #self.comp_out = nn.Linear(84,2)
        
        
    def forward(self,x):
        
        batch = x.shape[0]
        
        word_lstm,_ = self.word_lstm(x)
        
        q_w = self.q_w(word_lstm)
        k_w = self.k_w(word_lstm).permute(0,2,1)
        v_w = self.v_w(word_lstm)
        
        soft_att = self.softmax(torch.bmm(q_w,k_w))
        
        self_w_att = torch.bmm(soft_att,v_w)
        
        
        self_w_att = self_w_att.reshape(batch,-1,4,512)
        
        self_w_att = torch.mean(self_w_att,2)
        #self_w_att = torch.mean(word_lstm,1)
        
        sent_lstm,_ = self.sent_lstm(self.dropout(self_w_att))
        #sent_lstm,_ = self.sent_lstm(x)
        
        q_s = self.q_s(sent_lstm)
        k_s = self.k_s(sent_lstm).permute(0,2,1)
        v_s = self.v_s(sent_lstm)
        
        soft_att = self.softmax(torch.bmm(q_s,k_s))
        
        self_s_att = torch.bmm(soft_att,v_s)
        
        tweet_level = torch.mean(self_s_att,1)
        #tweet_level = torch.mean(word_lstm,1)
        
        #span_out = self.sigmoid(self.span_out(self.dropout(tweet_level)))
        
        #comp_fc = self.relu(self.comp_fc(self.dropout(tweet_level)))
        
        #final_rep = torch.cat((span_out,comp_fc),1)
        
        #comp_out = self.comp_out(final_rep)
        
        #x = x.permute(0,2,1)
        
        #conv = self.conv(x)
        
        #conv = conv.permute(0,2,1) 
        
        #conv_att = torch.mean(conv,1)
        
        final = self.dropout(tweet_level)
        
        #final = self_w_att*self.alphas_m1[0].expand_as(self_w_att) + conv_att*self.alphas_m2[0].expand_as(conv_att)
        #final = self_w_att + conv_att
        
        
        emo_fc2 = self.relu(self.emo_fc2(final))
        emo_out = self.relu(self.emo_out(emo_fc2))
        sen_fc2 = self.relu(self.sen_fc2(final))
        sen_out = self.relu(self.sen_out(sen_fc2))
        
        
        
        
       # comp_fc1 = self.relu(self.comp_fc1(x))
        
       # comp_fc2 = self.relu(self.comp_fc2(comp_fc1))
        comp_fc2 = self.relu(self.comp_fc2(final))
        #span_fc2 = self.relu(self.span_fc2(final))
        
        # central_b = emo_fc2*self.alphas_e[0].expand_as(emo_fc2) + sen_fc2*self.alphas_s[0].expand_as(sen_fc2) + comp_fc2*self.alphas_c[0].expand_as(comp_fc2)
        central_b = sen_fc2*self.alphas_s[0].expand_as(sen_fc2) + comp_fc2*self.alphas_c[0].expand_as(comp_fc2)
        #central_b = comp_fc2
        
        comp_fc3 = self.relu(self.comp_fc3(central_b))
        
        emo_cat = torch.cat((emo_out,torch.zeros(batch,121).to(device)),1)
        sen_cat = torch.cat((sen_out,torch.zeros(batch,125).to(device)),1)
        
        central_s = emo_cat*self.alphas_e[1].expand_as(emo_cat) + sen_cat*self.alphas_s[1].expand_as(sen_cat) + comp_fc3*self.alphas_c[1].expand_as(comp_fc3)
        # central_s = sen_cat*self.alphas_s[1].expand_as(sen_cat) + comp_fc3*self.alphas_c[1].expand_as(comp_fc3)
        #central_s = comp_fc3
        
        
        #rep = comp_fc2 + span_fc2
        
        #span_out = self.sigmoid(self.span_out(rep))
        span_out = self.sigmoid(self.span_out(central_s))
        
        #comp_out = self.relu(self.comp_out(comp_fc2))
        comp_out = self.relu(self.comp_out(central_s))
        
        
        
        
        
        return emo_out,sen_out,span_out,comp_out
        #return sen_out,sev_out,span_out,comp_out
        #return sev_out,span_out,comp_out
        
        
        
        #return span_out,comp_out

cuda


In [51]:

model = bert_span().to(device)
#model = lstm(dropout, emb_size,128, embeddings).to(device)

lr = 0.00001

combined_params =  list(model.parameters())

optimizer = torch.optim.Adam(combined_params, lr=lr, weight_decay=0) 

criterion_bce = nn.BCELoss() #Binary case
criterion_loss = nn.CrossEntropyLoss()

In [52]:
epochs = 20

exp_path = "coling2022"

In [56]:
def train_model(model, patience, n_epochs):

    epochs = n_epochs
#     clip = 5

    train_acc_list=[]
    val_acc_list=[]
    train_loss_list=[]
    val_loss_list=[]
    
 
    
        # initialize the experiment path
    Path(exp_path).mkdir(parents=True, exist_ok=True)
    # initialize early_stopping object
    
    model_file = os.path.join(exp_path, 'checkpoint_model.pt')
    
    early_stopping = EarlyStopping(patience=patience, verbose=True, path=model_file)

    model.train()

    for i in range(epochs):
        total_loss_train = 0
        total_train = 0
        correct_train = 0

        for data in dataloader_train:

            feat = data['feat'].to(device)
            label_train = data['label'].to(device)
            emotion_train = data['emotion'].to(device)
            sentiment_train = data['sentiment'].to(device)
            span_train = data['span'].to(device)
            #sarcasm_train = data['sarcasm'].to(device)
            
            
            #print(label_train.shape)
            
            model.zero_grad()
            
            #print(emotion_train.shape)
            
            
            #print(sent_level.shape)
            
            
            

            #span_out,output = model(doc)
            
            emo_out,sen_out,span_out,comp_out = model(feat)
            output = model(feat)
            
            span_loss = criterion_bce(span_out,span_train)
            bully_loss1 = criterion_loss(emo_out, label_train.long().squeeze(1))
            bully_loss2 = criterion_loss(sen_out, label_train.long().squeeze(1))
            bully_loss3 = criterion_loss(comp_out, label_train.long().squeeze(1))

            
            
            
            loss = span_loss +  bully_loss1 +  bully_loss2 + bully_loss3
            # loss = bully_loss 
            
            
            
            
            #print(loss)
            
            loss.backward()
            
            optimizer.step()
            
            with torch.no_grad():
                _, predicted_train = torch.max(comp_out.data, 1)
                #predicted_train = (output.data>=0.5).float()
                total_train += label_train.size(0)
                #total_train += emotion_train.size(0)
                
        
                correct_train += (predicted_train == label_train.squeeze(1)).sum().item()
                #correct_train += (predicted_train == emotion_train).sum().item()
    #                 out_val = (output.squeeze()>0.5).float()
    #                 out_final = ((out_val == 1).nonzero(as_tuple=True)[0])
    #                 print()
    #                 acc = torch.abs(output.squeeze() - label.float()).view(-1)
    #                 acc = (1. - acc.sum() / acc.size()[0])ret
    #                 total_acc_train += acc
                total_loss_train += loss.item()

        
        train_acc = 100 * correct_train / total_train
        train_loss = total_loss_train/total_train
        
        model.eval()
#         total_acc_val = 0
        total_loss_val = 0
        total_val = 0
        correct_val = 0

        with torch.no_grad():
            for data in dataloader_val:                
#                 Clip features...                

                
                feat = data['feat'].to(device)
                

                
                label_val = data['label'].to(device)
                emotion_val = data['emotion'].to(device)
                span_val = data['span'].to(device)
                
                sentiment_val = data['sentiment'].to(device)
                #sarcasm_val = data['sarcasm'].to(device)


              
                model.zero_grad()
               
                
              
                #span_out,output = model(doc)
                emo_out,sen_out,span_out,comp_out = model(feat)
                # output = model(feat)
               
                
                span_loss_val = criterion_bce(span_out,span_val)
              
                bully_loss_val1 = criterion_loss(emo_out, label_val.long().squeeze(1))
                bully_loss_val2 = criterion_loss(sen_out, label_val.long().squeeze(1))
                bully_loss_val3 = criterion_loss(comp_out, label_val.long().squeeze(1))

                
                val_loss = span_loss_val + bully_loss_val1 +  bully_loss_val2 + bully_loss_val3
                # val_loss = bully_loss_val 
                
                _, predicted_val = torch.max(comp_out.data, 1)
                
                
                total_val += label_val.size(0)
                #total_val += emotion_val.size(0)
                #_, label_val = torch.max(label_val, 1)
                correct_val += (predicted_val == label_val.squeeze(1)).sum().item()                
                #correct_val += (predicted_val == emotion_val).sum().item()                
                total_loss_val += val_loss.item()
                
                
        val_acc = 100 * correct_val / total_val
        val_loss = total_loss_val/total_val
        
        print(val_acc)

        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        
       
        early_stopping(val_loss, model)
        
        #print("Saving model...") 
        #torch.save(model.state_dict(), os.path.join(exp_path, "final.pt"))
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
            

            
        
        print(f'Epoch {i+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
        
       
        model.train()
        
        torch.cuda.empty_cache()
    
    # load the last checkpoint with the best model

    model.load_state_dict(torch.load(model_file))
    #model.load_state_dict(torch.load(os.path.join(exp_path,"final.pt")))
    
    return  model, train_acc_list, val_acc_list, train_loss_list, val_loss_list, i

In [59]:
def test_model(model):
    model.eval()
    
    
    total_test = 0
    correct_test =0
    total_acc_test = 0
    total_loss_test = 0
    outputs = []
    test_labels=[]
    with torch.no_grad():
        for data in dataloader_test:

            feat = data['feat'].to(device)
           
            label_test = data['label'].to(device)
            span_test = data['span'].to(device)
            
            
         
            #_,out = model(doc)
            # out = model(feat)
            emo_out,sen_out,span_out,comp_out = model(feat)

            
            
            
            outputs += list(comp_out.cpu().data.numpy())
            loss = criterion_loss(comp_out, label_test.squeeze(1).long())
            
            _, predicted_test = torch.max(comp_out.data, 1)
            #predicted_test = (out.data>=0.5).float()
            total_test += label_test.size(0)
            #_, label_test = torch.max(label_test, 1)
            correct_test += (predicted_test == label_test.squeeze(1)).sum().item()
#                 out_val = (output.squeeze()>0.5).float()
#                 out_final = ((out_val == 1).nonzero(as_tuple=True)[0])
#                 print()
#                 acc = torch.abs(output.squeeze() - label.float()).view(-1)
#                 acc = (1. - acc.sum() / acc.size()[0])
#                 total_acc_train += acc
            total_loss_test += loss.item()
            
            
#     #         print(label.float())
#             acc = torch.abs(out.squeeze() - label.float()).view(-1)
#     #         print((acc.sum() / acc.size()[0]))
#             acc = (1. - acc.sum() / acc.size()[0])
#     #         print(acc)
#             total_acc_test += acc
#             total_loss_test += loss.item()

    
    acc_test = 100 * correct_test / total_test
    loss_test = total_loss_test/total_test   
    
    print(f'acc: {acc_test:.4f} loss: {loss_test:.4f}')
    return outputs


In [60]:

n_epochs = 100
# early stopping patience; how long to wait after last time validation loss improved.
patience = 10
model , train_acc_list, val_acc_list, train_loss_list, val_loss_list, epoc_num = train_model(model, patience, n_epochs)

"""
span_file = os.path.join(exp_path, 'checkpoint_span.pt')
multitask_file = os.path.join(exp_path, 'checkpoint_mutlitask.pt')
abs_file = os.path.join(exp_path, 'checkpoint_absencoder.pt')

span_model.load_state_dict(torch.load(span_file))
multitask.load_state_dict(torch.load(multitask_file))
abs_encoder.load_state_dict(torch.load(abs_file))
"""

#chk_file = os.path.join(exp_path, 'checkpoint_'+exp_name+'.pt')
#model.load_state_dict(torch.load(chk_file))
# Plot the training and validation curves


outputs = test_model(model)


y_pred=[]
for i in outputs:
     y_pred.append(np.argmax(i))
# # np.argmax(outputs[:])
# outputs

print(len(y_pred))
# # Multiclass setting
test_labels=[]

for index in range(len(test["label"])):
    #test_labels.append(np.argmax(test["label"][index]))
    test_labels.append(test["label"][index])
    #test_labels.append(senti_label[test["sentiment"][index]])
    #test_labels.append(emoti_label[test["emotion"][index]])
    #test_labels.append(test["sarcasm"][index][0])
    
    
"""
for index, row in test_samples_frame.iterrows():
    lab = row['labels'][0]
    if lab=="not harmful":
        test_labels.append(0)
    elif lab=="somewhat harmful":
        test_labels.append(1)
    else:
        test_labels.append(2)
"""

# In[ ]:


def calculate_mmae(expected, predicted, classes):
    NUM_CLASSES = len(classes)
    count_dict = {}
    dist_dict = {}
    for i in range(NUM_CLASSES):
        count_dict[i] = 0
        dist_dict[i] = 0.0
    for i in range(len(expected)):
        dist_dict[expected[i]] += abs(expected[i] - predicted[i])
        count_dict[expected[i]] += 1
    overall = 0.0
    for claz in range(NUM_CLASSES): 
        class_dist =  1.0 * dist_dict[claz] / count_dict[claz] 
        overall += class_dist
    overall /= NUM_CLASSES
#     return overall[0]
    return overall


# In[ ]:


rec = np.round(recall_score(test_labels, y_pred, average="weighted"),4)
prec = np.round(precision_score(test_labels, y_pred, average="weighted"),4)
f1 = np.round(f1_score(test_labels, y_pred, average="weighted"),4)
# hl = np.round(hamming_loss(test_labels, y_pred),4)
acc = np.round(accuracy_score(test_labels, y_pred),4)
#mmae = np.round(calculate_mmae(test_labels, y_pred, [0,1]),4)
#mae = np.round(mean_absolute_error(test_labels, y_pred),4)
# print("recall_score\t: ",rec)
# print("precision_score\t: ",prec)
# print("f1_score\t: ",f1)
# print("hamming_loss\t: ",hl)
# print("accuracy_score\t: ",f1)

print(classification_report(test_labels, y_pred))


# In[ ]:


print("Acc, F1, Rec, Prec, MAE, MMAE")
#print(acc, f1, rec, prec, mae, mmae)
print(acc, f1, rec, prec)



rec = np.round(recall_score(test_labels, y_pred),4)
prec = np.round(precision_score(test_labels, y_pred),4)
f1 = np.round(f1_score(test_labels, y_pred),4)
# hl = np.round(hamming_loss(test_labels, y_pred),4)
acc = np.round(accuracy_score(test_labels, y_pred),4)

print(acc,f1,rec,prec) 

<ipython-input-48-5a7cdcf4f9f2>:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  feat=torch.tensor(self.data["feat"][idx].astype(np.float)).float().to(device)
<ipython-input-48-5a7cdcf4f9f2>:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  id=torch.tensor(self.data["id"][idx].astype(np.float)).long().to(device)
<ipython-input-48-5a7cdcf

84.84848484848484
Validation loss decreased (inf --> 0.069122).  Saving model ...
Epoch 1: train_loss: 0.0534 train_acc: 92.4747 | val_loss: 0.0691 val_acc: 84.8485
84.17508417508418
EarlyStopping counter: 1 out of 10
Epoch 2: train_loss: 0.0539 train_acc: 91.9506 | val_loss: 0.0694 val_acc: 84.1751
83.83838383838383
EarlyStopping counter: 2 out of 10
Epoch 3: train_loss: 0.0539 train_acc: 91.7259 | val_loss: 0.0694 val_acc: 83.8384
79.46127946127946
EarlyStopping counter: 3 out of 10
Epoch 4: train_loss: 0.0543 train_acc: 91.5387 | val_loss: 0.0753 val_acc: 79.4613
80.47138047138047
EarlyStopping counter: 4 out of 10
Epoch 5: train_loss: 0.0521 train_acc: 93.0738 | val_loss: 0.0742 val_acc: 80.4714
81.81818181818181
EarlyStopping counter: 5 out of 10
Epoch 6: train_loss: 0.0510 train_acc: 94.0097 | val_loss: 0.0729 val_acc: 81.8182
81.48148148148148
EarlyStopping counter: 6 out of 10
Epoch 7: train_loss: 0.0509 train_acc: 93.6728 | val_loss: 0.0741 val_acc: 81.4815
81.81818181818181
E